# OpenVINO example with Squeezenet Model

This notebook illustrates how you can serve [OpenVINO](https://software.intel.com/en-us/openvino-toolkit) optimized models for Imagenet with Seldon Core.

<img src="dog.jpeg"/>

## Dependencies

 * Seldon-core (```pip install seldon-core```)
 * Numpy
 * Keras
 * Matplotlib
 * Tensorflow

## Download Squeezenet Model

We will download a pre-trained and optimized model for OpenVINO CPU into a local folder.

In [ ]:
!mkdir -p models/squeezenet/1 && \
    wget -O models/squeezenet/1/squeezenet1.1.xml https://s3-eu-west-1.amazonaws.com/seldon-public/openvino-squeeznet-model/squeezenet1.1.xml && \
    wget -O models/squeezenet/1/squeezenet1.1.mapping https://s3-eu-west-1.amazonaws.com/seldon-public/openvino-squeeznet-model/squeezenet1.1.mapping && \
    wget -O models/squeezenet/1/squeezenet1.1.bin https://s3-eu-west-1.amazonaws.com/seldon-public/openvino-squeeznet-model/squeezenet1.1.bin 

## Run Seldon Core on Minikube

**The example below assumes Minikube 0.30.0 installed**

In [ ]:
!minikube start --memory 4096 --disk-size 20g --extra-config=apiserver.authorization-mode=RBAC

In [ ]:
!kubectl create namespace seldon

In [ ]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

In [ ]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

In [ ]:
!helm init

In [ ]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

In [ ]:
!helm install ../../../helm-charts/seldon-core-crd --name seldon-core-crd  --set usage_metrics.enabled=true
!helm install ../../../helm-charts/seldon-core --name seldon-core --set ambassador.enabled=true

## Mount local folder onto minikube for HostPath
Run in the current folder:
```
minikube mount ./models:/opt/ml
```

This will allow the model folder containing the Squeezenet model to be accessed. For production deployments you would use a NFS volume.

## Build Combiner Image

In [ ]:
!eval $(minikube docker-env) && cd resources/combiner && s2i build -E environment_grpc . seldonio/seldon-core-s2i-python36:0.4 seldonio/imagenet_combiner:0.1

In [ ]:
!eval $(minikube docker-env) && cd resources/transformer && s2i build -E environment_grpc . seldonio/seldon-core-s2i-python36:0.4 seldonio/imagenet_transformer:0.1

## Deploy Seldon Intel OpenVINO Graph

In [ ]:
import sys
sys.path.append("../../../notebooks")
from visualizer import *

In [ ]:
get_graph("seldon_openvino_ensemble.json")

In [ ]:
!pygmentize seldon_openvino_ensemble.json

In [ ]:
!kubectl apply -f pvc.json
!kubectl apply -f seldon_openvino_ensemble.json

# Serve Requests

**Ensure you port forward ambassador:**

```
kubectl port-forward $(kubectl get pods -n seldon -l service=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080
```

In [ ]:
import tensorflow as tf
from seldon_core.proto import prediction_pb2
from seldon_core.proto import prediction_pb2_grpc
import grpc

def grpc_request_ambassador(deploymentName,namespace,endpoint="localhost:8004",data=None):
    datadef = prediction_pb2.DefaultData(
                names = 'x',
                tftensor = tf.make_tensor_proto(data)
            )
    request = prediction_pb2.SeldonMessage(data = datadef)
    channel = grpc.insecure_channel(endpoint)
    stub = prediction_pb2_grpc.SeldonStub(channel)
    if namespace is None:
        metadata = [('seldon',deploymentName)]
    else:
        metadata = [('seldon',deploymentName),('namespace',namespace)]
    response = stub.Predict(request=request,metadata=metadata)
    return response


In [ ]:
%matplotlib inline
import numpy as np
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.preprocessing import image
import sys
import json
import matplotlib.pyplot as plt
import datetime
API_AMBASSADOR="localhost:8003"

def getImage(path):
    img = image.load_img(path, target_size=(227, 227))
    x = image.img_to_array(img)
    plt.imshow(x/255.)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

def getImageRaw(path):
    img = image.load_img(path, target_size=(227, 227))
    x = image.img_to_array(img)
    plt.imshow(x/255.)
    return x

#X = getImage("car.png")
#X = X.transpose((0,3,1,2))

#X = getImageRaw("car.png")
X = getImageRaw("dog.jpeg")
print(X.shape)
start_time = datetime.datetime.now()
response = grpc_request_ambassador("openvino-model","seldon",API_AMBASSADOR,data=X)
end_time = datetime.datetime.now()
duration = (end_time - start_time).total_seconds() * 1000
print(duration)

print(response.strData)

#result = tf.make_ndarray(response.data.tftensor)   
#result = result.reshape(1,1000)

#with open('imagenet_classes.json') as f:
#    cnames = eval(f.read())
#
#    single_result = result[[0],...]
#    ma = np.argmax(single_result)
#    print("\t",0, cnames[ma])


In [ ]:
durations = []
for i in range(100):
    start_time = datetime.datetime.now()
    response = grpc_request_ambassador("openvino-model","seldon",API_AMBASSADOR,data=X)
    end_time = datetime.datetime.now()
    duration = (end_time - start_time).total_seconds() * 1000
    durations.append(duration)
print(sum(durations) / float(len(durations)))